# Transfer learning, finetune VGG16

## 1. Why VGG16

VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper "Very Deep Convolutional Networks for Large-Scale Image Recognition". It is one of the most popular pre-trained models for image classification tasks.

### Key Features of VGG16
1. Architecture: VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper "Very Deep Convolutional Networks for Large-Scale Image Recognition".<br>
2. Layers: It has 16 layers with learnable weights, including 13 convolutional layers and 3 fully connected layers.<br>
3. Pre-trained Weights: The model is pre-trained on the ImageNet dataset, which contains 1.2 million images and 1000 classes.<br>
4. Usage: It is widely used for image classification tasks and can be fine-tuned for specific tasks such as brain tumor classification.

## 2. Finetune VGG16 for brain tumor classification task

### 2.1 Preprocessing data

In [2]:
import cv2
import os
import random
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import normalize
from PIL import Image
from sklearn.model_selection import train_test_split

In [3]:
seed = 99
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

#### Reading and nomalize dataset

In [4]:
no_dir = os.listdir('./data_no/data_no/NO/')
yes_dir = os.listdir('./data_yes/data_yes/YES/')
data_set,label = [],[]
for i,cur_img_dir in enumerate(no_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_no/data_no/NO/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(0)
for i,cur_img_dir in enumerate(yes_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_yes/data_yes/YES/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(1)

In [10]:
data_set = np.array(data_set)
label = np.array(label)

#### Splitting the data into training and testing

In [19]:
x_train,x_test,y_train,y_test = train_test_split(
    data_set,label,
    test_size=0.2,
    random_state=99
    )


In [21]:
print(f'X train shape: {x_train.shape}\nY train shape: {y_train.shape}\nX test shape: {x_test.shape}\nY test shape: {y_test.shape})')

X train shape: (873, 64, 64, 3)
Y train shape: (873,)
X test shape: (219, 64, 64, 3)
Y test shape: (219,))


#### Nomalize

In [13]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)
x_val = normalize(x_val, axis=1)

### Loading pre-trained model

In [15]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [25]:
# Load the VGG16 model with pre-trained weights, excluding the top fully connected layers
base_model = VGG16(
    weights='imagenet', 
    include_top=False, 
    input_shape=(64, 64, 3)
    )

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False


In [26]:
# Add custom top layers for classification
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x) #binary classification 


In [27]:
# Create the new model
model = Model(inputs=base_model.input, outputs=x)

In [28]:
model.compile(
    optimizer=Adam(lr=1e-4, amsgrad=True),
    loss='binary_crossentropy', 
    metrics=['accuracy']
    )
# Train the model
history = model.fit(
    x_train, 
    y_train, 
    epochs=10, 
    batch_size=32, 
    validation_data = (x_test, y_test)
    )

Epoch 1/10



28/28 [==============================] - 21s 677ms/step - loss: 1.4543 - accuracy: 0.8110 - val_loss: 0.2840 - val_accuracy: 0.9315
Epoch 2/10
28/28 [==============================] - 18s 644ms/step - loss: 0.2645 - accuracy: 0.9278 - val_loss: 0.4195 - val_accuracy: 0.8995
Epoch 3/10
28/28 [==============================] - 18s 648ms/step - loss: 0.2449 - accuracy: 0.9313 - val_loss: 0.1887 - val_accuracy: 0.9452
Epoch 4/10
28/28 [==============================] - 19s 681ms/step - loss: 0.1121 - accuracy: 0.9668 - val_loss: 0.2459 - val_accuracy: 0.9132
Epoch 5/10
28/28 [==============================] - 18s 660ms/step - loss: 0.0819 - accuracy: 0.9668 - val_loss: 0.1638 - val_accuracy: 0.9498
Epoch 6/10
28/28 [==============================] - 19s 695ms/step - loss: 0.0351 - accuracy: 0.9851 - val_loss: 0.1756 - val_accuracy: 0.9452
Epoch 7/10
28/28 [==============================] - 20s 726ms/step - loss: 0.0424 - accuracy: 0.9828 - val_loss: 0.3485 - val_accuracy: 0.9041
Epoch 8/10